# Treinamento e validação de Modelo

In [6]:
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler
from scipy.sparse import csr_matrix, hstack
from python_scripts.save_load import load_df_from_bucket, save_df_to_s3_bucket, save_sparse_vector_to_s3_bucket_as_libsvm

### Processamento adicional
Para utilizar os dados obtidos para treinamento de um modelo, algumas etapas adicionais de processamento serão executadas:
- Apenas as colunas de lemmas, tamanho médio de sentença e tamanho médio de palavra serão utilizadas no modelo
- Separação de dados de treino, validação e teste
- Transformação TF-IDF será aplicada à coluna de lemmas
- Colunas de tamanhos de sentenças e palavras serão padronizadas (ie RobustScaler para evitar problemas com outliers)
- Salvar arquivos referentes aos dados de treino, validação e teste após este processamento

In [7]:
full_df = load_df_from_bucket('dados_processados.csv', tipo='processado')
full_df.head()

,fake,text,words,lemmas,avg_sent_len,avg_word_len,words_str,lemmas_str
0,0,A divisão do STF ao meio entre partidários e ...,"['divisão', 'STF', 'meio', 'partidários', 'ind...","['divisão', 'STF', 'meio', 'partidário', 'inde...",10.747664,6.690641,divisão STF meio partidários independentes fic...,divisão STF meio partidário independente ficar...
1,1,"General manda recado para STF: ""Abaixaram as c...","['General', 'manda', 'recado', 'STF', 'Abaixar...","['general', 'mandar', 'recado', 'STF', 'abaixa...",11.000000,6.461584,General manda recado STF Abaixaram calças Cong...,general mandar recado STF abaixar calça congre...
2,1,O Nordeste acordou! Lula e o PT são enxotados:...,"['Nordeste', 'acordou', 'Lula', 'PT', 'enxotad...","['nordeste', 'acordar', 'Lula', 'PT', 'enxotar...",7.333333,6.565873,Nordeste acordou Lula PT enxotados Chega bande...,nordeste acordar Lula PT enxotar chegar bandei...
3,0,"Dois relatórios da Polícia Federal, com análi...","['Dois', 'relatórios', 'Polícia', 'Federal', '...","['dois', 'relatório', 'Polícia', 'Federal', 'a...",17.000000,7.237319,Dois relatórios Polícia Federal análises mater...,dois relatório Polícia Federal análise materia...
4,1,Coreia do Norte declara status de QUASE-GUERRA...,"['Coreia', 'Norte', 'declara', 'status', 'QUAS...","['Coreia', 'Norte', 'declarar', 'status', 'QUA...",11.666667,6.513799,Coreia Norte declara status QUASE-GUERRA mobil...,Coreia Norte declarar status QUASE-GUERRA mobi...


In [9]:
model_df = full_df[['fake', 'lemmas_str', 'avg_sent_len', 'avg_word_len']]
model_df.head()

,fake,lemmas_str,avg_sent_len,avg_word_len
0,0,divisão STF meio partidário independente ficar...,10.747664,6.690641
1,1,general mandar recado STF abaixar calça congre...,11.000000,6.461584
2,1,nordeste acordar Lula PT enxotar chegar bandei...,7.333333,6.565873
3,0,dois relatório Polícia Federal análise materia...,17.000000,7.237319
4,1,Coreia Norte declarar status QUASE-GUERRA mobi...,11.666667,6.513799


In [10]:
train, test_validate = train_test_split(model_df, test_size=0.2, random_state=42, stratify=model_df['fake'])
test, validate = train_test_split(test_validate, test_size=0.2, random_state=42, stratify=test_validate['fake'])

### Processamento adicional

Processamento adicional feito via ColumnTransformer do Scikit-learn, para que se aplique Scaling apenas às colunas numéricas e se aplique TFIDF apenas às colunas textuais. O transformador é fittado apenas nos dados de treinamento, mas a transformação é aplicada nos três conjuntos (treinamento, validação e teste)

In [11]:
# Criar vetorizador TFIDF e ajustar aos dados de treinamento
tfidf = TfidfVectorizer(lowercase=False, ngram_range = (1,2))
tfidf.fit(train['lemmas_str'])

TfidfVectorizer(lowercase=False, ngram_range=(1, 2))

In [12]:
def create_sparse(df, tfidf_model):
    """
    Função auxiliar para criar matriz esparsa com a vectorização TF-IDF adicionada de outras
    colunas relevantes de df (fake, avg_sent_len e avg_word_len)
    Coluna 'fake' vem primeiro, padrão do SageMaker
    
    Notes
    -----
        Enviar tfidf_model já treinado, aqui usa apenas transform!!
    """
    sparse_vector = tfidf_model.transform(df['lemmas_str'])
    df_no_lemmas = df.drop(['lemmas_str', 'fake'], axis=1)
    df_as_sparse = csr_matrix(df_no_lemmas.values)
    final_sparse = hstack([df_as_sparse, sparse_vector])
#     return final_sparse, df['fake'].values
    return df_as_sparse, df['fake'].values   # Debugando, usar só as colunas numéricas pra ficar levim

In [13]:
x_train, y_train = create_sparse(train, tfidf)
x_validate, y_validate = create_sparse(validate, tfidf)
x_test, y_test = create_sparse(test, tfidf)

### Salvar matrizes esparsas no S3 para utilizar nos modelos
Pontos importantes para funcionar com treinamento do Sagemaker:
- Coluna alvo ('fake') deve ser a primeira coluna do DataFrame (já está neste formato)
- Salvar sem cabeçalhos nem índices (já está feito, são matrizes esparsas puras)

In [38]:
sparse_name_tuples = [(x_train, y_train, 'train.libsvm'), 
                      (x_test, y_test, 'test.libsvm'), 
                      (x_validate, y_validate, 'validate.libsvm')]

for x, y, file_name in sparse_name_tuples:
    save_sparse_vector_to_s3_bucket_as_libsvm(x, y, file_name, tipo='modelo')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/amazon

# Treinar modelo

In [1]:
import sagemaker
import boto3
from python_scripts.save_load import BUCKET_MODEL

In [2]:
container = sagemaker.image_uris.retrieve('xgboost',boto3.Session().region_name,'1.0-1')

hyperparams={"num_round":"42",
             "eval_metric": "auc",
             "objective": "binary:logistic"}

s3_output_location=f"s3://{BUCKET_MODEL}/output/"

In [3]:
xgb_model=sagemaker.estimator.Estimator(container,
                                        sagemaker.get_execution_role(),
                                        instance_count=1,   # Mudar para algo maior que 1 no definitivo
                                        instance_type='ml.m4.xlarge',
                                        output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())

train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/{}".format(BUCKET_MODEL,'train','train.libsvm'), content_type='libsvm')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/{}".format(BUCKET_MODEL,'validate','validate.libsvm'), content_type='libsvm')

data_channels = {'train': train_channel, 'validation': validate_channel}

# Deu erro por diferença em nome de features!
xgb_model.fit(inputs=data_channels)

print('ready for hosting!')

2022-11-11 11:14:28 Starting - Starting the training job...ProfilerReport-1668165268: InProgress
...
2022-11-11 11:15:19 Starting - Preparing the instances for training.........
2022-11-11 11:16:59 Downloading - Downloading input data...
2022-11-11 11:17:25 Training - Downloading the training image...
2022-11-11 11:18:00 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value auc to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
[11:17:55] 5760x2 matrix with 11520 entries loaded from /opt/ml/input/data/train
[11:17:55] 288x2 matrix with 576 entries loaded

In [46]:
#tbm existe serializer=sagemaker.amazon.common.RecordSerializer(),

xgb_predictor = xgb_model.deploy(initial_instance_count=1,
                                 serializer=sagemaker.serializers.LibSVMSerializer(),
                                 instance_type='ml.m4.xlarge')

-------!

In [48]:
import s3fs
fs = s3fs.S3FileSystem()
serializer = sagemaker.serializers.LibSVMSerializer()
s3_path = 's3://projetointerdisciplinartreinoteste/test/test.libsvm'

with fs.open(s3_path) as libsvm_file:
    y_pred = xgb_predictor.predict(libsvm_file)

In [57]:
y_pred_list = [1 if float(x) >= 0.5 else 0 for x in y_pred.decode('utf-8').split(',')]


In [60]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.69      0.75      0.72       576
           1       0.72      0.66      0.69       576

    accuracy                           0.71      1152
   macro avg       0.71      0.71      0.71      1152
weighted avg       0.71      0.71      0.71      1152



In [ ]:
# Funcionou com csvserializer, mas não é o ideal...

import io
batch_X_csv_buffer = io.StringIO()
test[['avg_sent_len', 'avg_word_len']].to_csv(batch_X_csv_buffer, header=False, index=False)
test_row = batch_X_csv_buffer.getvalue()

In [37]:
from sagemaker.amazon.common import write_spmatrix_to_sparse_tensor
import io
buf = io.BytesIO()
write_spmatrix_to_sparse_tensor(buf,x_test)
buf.seek(0)

TypeError: string argument expected, got 'bytes'

In [36]:
y_pred = xgb_predictor.predict('s3://projetointerdisciplinartreinoteste/test/test.io')
y_pred

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [39]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)  # deletar para não ficar cobrando

In [26]:
from sklearn.datasets import dump_svmlight_file

dump_svmlight_file(x_train, y_train, 'train.libsvm')

In [22]:
x_train.shape

(4696, 1023884)

In [24]:
y_train.shape

(4696,)

In [23]:
x_validate.shape

(235, 1023884)

In [25]:
y_validate.shape

(235,)